In [8]:
import wordsearch as ws
from IPython.display import HTML, display, FileLink

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
wordlist = ['camel','tunnel','squirrel','travel','towel','tinsel','bagel','hazel','vowel','jewel','cry','fly','dry','try','reply','sly','shy','terrify','sky','multiply']

In [10]:
ws.make_wordsearch_pack(wordlist, "11th December 2020: Spelling Wordsearch ")

In [11]:
display(FileLink("easy.html")) 

C:\Users\boylesa\Desktop\Wordsearch\easy.html

In [12]:
display(FileLink("medium.html"))

C:\Users\boylesa\Desktop\Wordsearch\medium.html

In [13]:
display(FileLink("hard.html"))

C:\Users\boylesa\Desktop\Wordsearch\hard.html

In [14]:
display(FileLink("answers.txt"))

C:\Users\boylesa\Desktop\Wordsearch\answers.txt